In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape, Conv2D, BatchNormalization, Activation, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar10

# Load and Preprocess the Data
def load_and_preprocess_data(add_noise=True, noise_factor=0.2):
    (x_train, _), (x_test, _) = cifar10.load_data()
    x_train = x_train.astype('float32') / 255.0
    x_test = x_test.astype('float32') / 255.0
    print("Training data shape:", x_train.shape)
    print("Test data shape:", x_test.shape)
    
    x_train_noisy = x_train.copy()
    x_test_noisy = x_test.copy()
    
    # Add random Gaussian noise if requested
    if add_noise:
        x_train_noisy = x_train_noisy + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
        x_test_noisy = x_test_noisy + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)
        x_train_noisy = np.clip(x_train_noisy, 0.0, 1.0)
        x_test_noisy = np.clip(x_test_noisy, 0.0, 1.0)
    
    return x_train_noisy, x_train, x_test_noisy, x_test

# MLP Autoencoder
def build_autoencoder(input_shape=(32, 32, 3), encoding_dim=256):
    input_dim = np.prod(input_shape)
    input_img = Input(shape=input_shape)
    flat_img = Flatten()(input_img)
    encoded = Dense(512, activation='relu')(flat_img)
    encoded = Dense(encoding_dim, activation='relu')(encoded)
    decoded = Dense(512, activation='relu')(encoded)
    decoded = Dense(input_dim, activation='sigmoid')(decoded)
    output_img = Reshape(input_shape)(decoded)
    
    autoencoder = Model(input_img, output_img)
    encoder = Model(input_img, encoded)
    
    encoded_input = Input(shape=(encoding_dim,))
    decoder_layers = autoencoder.layers[-3](encoded_input)
    decoder_layers = autoencoder.layers[-2](decoder_layers)
    decoder_output = Reshape(input_shape)(decoder_layers)
    decoder = Model(encoded_input, decoder_output)
    
    return autoencoder, encoder, decoder

# Convolutional Autoencoder
def build_convolutional_autoencoder(input_shape=(32, 32, 3)):
    # Input layer
    input_img = Input(shape=input_shape)
    
    # Encoder
    x = Conv2D(32, (3, 3), padding='same')(input_img)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)  # 16x16x32
    
    x = Conv2D(64, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)  # 8x8x64
    
    x = Conv2D(128, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)  # 4x4x128
    
    # Decoder
    x = Conv2D(128, (3, 3), padding='same')(encoded)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = UpSampling2D((2, 2))(x)  # 8x8x128
    
    x = Conv2D(64, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = UpSampling2D((2, 2))(x)  # 16x16x64
    
    x = Conv2D(32, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = UpSampling2D((2, 2))(x)  # 32x32x32
    
    decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)  # 32x32x3
    
    # Create the autoencoder model
    autoencoder = Model(input_img, decoded)
    # Create the encoder model
    encoder = Model(input_img, encoded)
    # Create the decoder model
    encoded_input = Input(shape=(4, 4, 128))
    # Get the decoder layers from the autoencoder
    decoder_layers = autoencoder.layers[-13:]
    # Build the decoder model
    x = encoded_input
    for layer in decoder_layers:
        x = layer(x)
    decoder = Model(encoded_input, x)
    
    return autoencoder, encoder, decoder

# Train Autoencoder
def train_autoencoder(autoencoder, x_train_noisy, x_train, x_test_noisy, x_test, batch_size=128, epochs=50):
    # Compile the model
    autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    # Use early stopping to prevent overfitting
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    # Train the model
    history = autoencoder.fit(
        x_train_noisy, x_train,
        epochs=epochs,
        batch_size=batch_size,
        shuffle=True,
        validation_data=(x_test_noisy, x_test),
        callbacks=[early_stopping]
    )
    return history

# Evaluate Autoencoder
def evaluate_autoencoder(autoencoder, x_test_noisy, x_test):
    test_loss = autoencoder.evaluate(x_test_noisy, x_test)
    print(f"Test loss (MSE): {test_loss}")
    return test_loss

# Visualize Reconstructed Images
def plot_reconstructed_images(autoencoder, x_test_noisy, x_test, n=10):
    reconstructed_imgs = autoencoder.predict(x_test_noisy[:n])
    plt.figure(figsize=(20, 4))
    for i in range(n):
        # Original image
        ax = plt.subplot(3, n, i + 1)
        plt.imshow(x_test[i])
        plt.title("Original")
        plt.axis("off")
        # Noisy image
        ax = plt.subplot(3, n, i + n + 1)
        plt.imshow(x_test_noisy[i])
        plt.title("Noisy")
        plt.axis("off")
        # Reconstructed image
        ax = plt.subplot(3, n, i + 2*n + 1)
        plt.imshow(reconstructed_imgs[i])
        plt.title("Reconstructed")
        plt.axis("off")
    plt.tight_layout()
    plt.show()

# Main Execution
if __name__ == "__main__":
    # Load and preprocess data
    x_train_noisy, x_train, x_test_noisy, x_test = load_and_preprocess_data(add_noise=True, noise_factor=0.2)
    
    # Build MLP Autoencoder
    autoencoder_mlp, encoder_mlp, decoder_mlp = build_autoencoder()
    
    # Train MLP Autoencoder
    history_mlp = train_autoencoder(autoencoder_mlp, x_train_noisy, x_train, x_test_noisy, x_test)
    
    # Evaluate MLP Autoencoder
    evaluate_autoencoder(autoencoder_mlp, x_test_noisy, x_test)
    
    # Visualize MLP Autoencoder results
    plot_reconstructed_images(autoencoder_mlp, x_test_noisy, x_test)
    
    # Build Convolutional Autoencoder
    autoencoder_conv, encoder_conv, decoder_conv = build_convolutional_autoencoder()
    
    # Train Convolutional Autoencoder
    history_conv = train_autoencoder(autoencoder_conv, x_train_noisy, x_train, x_test_noisy, x_test)
    
    # Evaluate Convolutional Autoencoder
    evaluate_autoencoder(autoencoder_conv, x_test_noisy, x_test)
    
    # Visualize Convolutional Autoencoder results
    plot_reconstructed_images(autoencoder_conv, x_test_noisy, x_test)

Training data shape: (50000, 32, 32, 3)
Test data shape: (10000, 32, 32, 3)
Epoch 1/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 36s 81ms/step - loss: 0.0396 - val_loss: 0.0200
Epoch 2/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 31s 78ms/step - loss: 0.0179 - val_loss: 0.0147
Epoch 3/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 31s 79ms/step - loss: 0.0143 - val_loss: 0.0132
Epoch 4/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 40s 77ms/step - loss: 0.0131 - val_loss: 0.0125
Epoch 5/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 40s 75ms/step - loss: 0.0123 - val_loss: 0.0119
Epoch 6/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 30s 76ms/step - loss: 0.0116 - val_loss: 0.0112
Epoch 7/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 42s 79ms/step - loss: 0.0112 - val_loss: 0.0110
Epoch 8/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 41s 78ms/step - loss: 0.0107 - val_loss: 0.0105
Epoch 9/50
 81/391 ━━━━━━━━━━━━━━━━━━━━ 21s 71ms/step - loss: 0.0106

KeyboardInterrupt: 